In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense

from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dropout, Dense

from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
# from keras.utils import to_categorical

import cv2

import os

In [ ]:
root_path = '/content/car_and_bike/'
class_names = sorted(os.listdir(root_path))
n_classes = len(class_names)

# Class Distribution
class_dis = [len(os.listdir(root_path + name)) for name in class_names]


# Show
print(f"Total Number of Classes : {n_classes} \nClass Names : {class_names}")

In [ ]:
# Initialize Generator
BATCH_SIZE = 32
IMGSIZE = 244
train_gen = ImageDataGenerator(rescale=1/255., rotation_range=10, validation_split=0.2)

# Load Data
train_ds = train_gen.flow_from_directory(root_path, class_mode='binary', target_size=(IMGSIZE,IMGSIZE), shuffle=True, batch_size=32, subset='training')
valid_ds = train_gen.flow_from_directory(root_path, class_mode='binary', target_size=(IMGSIZE,IMGSIZE), shuffle=True, batch_size=32, subset='validation') 

In [ ]:
j = np.random.randint(BATCH_SIZE)
X, y = valid_ds[j]

fig, ax = plt.subplots(5, 4, figsize=(20, 16))

for i, axis in enumerate(ax.flat):
    axis.imshow(X[i])
    bike = class_names[int(y[i])]
    axis.set(title=bike)

In [ ]:
mobilenet = MobileNet(input_shape=(244, 244, 3) ,weights='imagenet', include_top=False)

In [ ]:
class VGG16(tnn.Module):
    def __init__(self, n_classes=1000):
        super(VGG16, self).__init__()

        # Build Up Architecture
        # Conv blocks (BatchNorm + ReLU activation added in each block)
        self.layer1 = vgg_conv_block([3, 64], [64, 64], [3, 3], [1, 1], 2, 2)
        self.layer2 = vgg_conv_block([64, 128], [128, 128], [3, 3], [1, 1], 2, 2)
        self.layer3 = vgg_conv_block([128, 256, 256], [256, 256, 256], [3, 3, 3], [1, 1, 1], 2, 2)
        self.layer4 = vgg_conv_block([256, 512, 512], [512, 512, 512], [3, 3, 3], [1, 1, 1], 2, 2)
        self.layer5 = vgg_conv_block([512, 512, 512], [512, 512, 512], [3, 3, 3], [1, 1, 1], 2, 2)

        # FC layers
        self.layer6 = vgg_fc_layer(7 * 7 * 512, 4096)
        self.layer7 = vgg_fc_layer(4096, 4096)

        # Final layer
        self.layer8 = tnn.Linear(4096, n_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        vgg16_features = self.layer5(out)
        out = vgg16_features.view(out.size(0), -1)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)

        return vgg16_features, out


vgg16 = VGG16(n_classes=N_CLASSES)
vgg16.cuda()

In [ ]:
# image-input layer
inputs = Input(shape=(IMGSIZE, IMGSIZE, 3))

# define the full network
x = mobilenet(inputs) # base-model
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x) # add dense layers so model learn more complex functions
x = Dropout(0.5)(x) # to avoid overfitting
x = Dense(256, activation='relu')(x) # dense layer 2

# output layer
outputs = Dense(1, activation='sigmoid')(x)

# set the model
model = Model(inputs = inputs, outputs=outputs)

# freeze base-model (mobilenet) weights
model.layers[1].trainable = False

model.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])

# model summary
model.summary()

In [ ]:
history = model.fit_generator(train_ds, 
                              epochs=5, 
                              validation_data = valid_ds, 
                             )

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

# summarize history for accuracy
plt.figure(figsize=(15, 5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()